In [1]:
# Scrapping from reclameaqui.com.br
import pandas as pd
import time
import random
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager
import os 
from selenium.webdriver.common.by import By
# import undetected_chromedriver as uc


In [2]:
# nome da empresa seguindo convenção do site reclameaqui.com.br
empresa : str = 'banco-do-brasil'

# URL da pagina principal da empresa
url_p = 'https://www.reclameaqui.com.br/empresa/' + empresa + '/'

# URL da pagina de reclamações da empresa
def url_r(page : int, type : str = 'EVALUATED') -> str:

    # https://www.reclameaqui.com.br/empresa/temu/lista-reclamacoes/?pagina=1&status=EVALUATED
    return 'https://www.reclameaqui.com.br/empresa/' + empresa + '/lista-reclamacoes/?pagina='+ str(page) +'&status=' + type



In [18]:
# pegando metadados da empresa (nome, categoria, reputação, sobre)

# driver
firefox_options = Options()
firefox_options.add_argument("--start-maximized")  # Start maximized (not always supported)
#firefox_options.add_argument("--headless")  # Run headless (optional)
service = Service(GeckoDriverManager().install())
driver = webdriver.Firefox(service=service, options=firefox_options)

In [ ]:

# acessando a pagina principal da empresa
driver.get(url_p)

time.sleep(random.randint(4,6))
   
# ramo de atividade da empresa
categoria = driver.find_element(By.ID, 'info_segmento_hero').text

# reputação da empresa
reputacao = driver.find_element(By.ID, 'tag_reputacao_hero').text

# sobre a empresa
sobre = driver.find_element(By.CSS_SELECTOR, 'ul.sc-1915fv4-2.idgfcC').text
sobre = sobre.split("\n")[0] 

metadata = {
   'nome': empresa,
   'categoria': categoria,         
   'reputacao': reputacao,         
   'sobre': sobre       
}


print(metadata)


{'nome': 'banco-do-brasil', 'categoria': 'Bancos e Financeiras', 'reputacao': 'Bom', 'sobre': 'O BB é um banco de economia mista e a maior instituição financeira do país, com mais de 210 anos de história. Possui cerca de 5 mil agências localizadas na maioria dos municípios brasileiros.'}


In [230]:
# pegando hyperlinks das reclamações respondidas

driver.get(url_r(page=1, type='EVALUATED'))

all_hyperlinks = []

Numero_de_paginas = 50 # máximo de 50 páginas

for i in range(Numero_de_paginas):
   try:
      div_hyperlinks = driver.find_elements(By.CSS_SELECTOR, 'div.sc-1pe7b5t-0.eJgBOc') # div dos hyperlinks das reclamações
      
      hyperlinks = [h.find_element(By.CSS_SELECTOR, 'a').get_attribute('href') for h in div_hyperlinks] # pegando os hyperlinks
      
      print(hyperlinks)
      
      all_hyperlinks.extend(hyperlinks)
      
      botao = driver.find_element(By.CSS_SELECTOR, "button[data-testid='next-page-navigation-button']")
      driver.execute_script("arguments[0].click();", botao)

      time.sleep(random.randint(5, 8))
      
   except:
      
      break
   
   
print(all_hyperlinks)
   

['https://www.reclameaqui.com.br/banco-do-brasil/cobranca-anuidade_Lk-6AKyO3vU8Cq0p/', 'https://www.reclameaqui.com.br/banco-do-brasil/demora-em-reconhecer-pagamento_f9jqZrP4JrtDNuAC/', 'https://www.reclameaqui.com.br/banco-do-brasil/anuidade-cartao_4s9HVo3RsytfvmrF/', 'https://www.reclameaqui.com.br/banco-do-brasil/consignado-recusado-diversas-vezes_7q--vrnhUVTnxSrA/', 'https://www.reclameaqui.com.br/banco-do-brasil/conta-limitada_JnZAp_2CwwTwqFQH/', 'https://www.reclameaqui.com.br/banco-do-brasil/informe-de-rendimentos-escriturais_iUGxLmfu_XFxsj8q/', 'https://www.reclameaqui.com.br/banco-do-brasil/cobranca-tributos-irregular_cs6Fv2-fYtL2qkf3/', 'https://www.reclameaqui.com.br/banco-do-brasil/envio-de-salario-para-o-banco-indevido_zNSlMbKmmLEbLRAd/', 'https://www.reclameaqui.com.br/banco-do-brasil/numero-clonado-do-banco-do-brasil_6XDjhyATvwqxqZTc/', 'https://www.reclameaqui.com.br/banco-do-brasil/colocou-meu-nome-no-serasa_AdZqk52AkjSpRa9l/']
['https://www.reclameaqui.com.br/banco-do

In [ ]:
# pegando dados das reclamações respondidas

reclamacoes = []
#for link in all_hyperlinks:
for link in ['https://www.reclameaqui.com.br/banco-do-brasil/demora-em-reconhecer-pagamento_f9jqZrP4JrtDNuAC/']:
   
   try:
      driver.get(link)

      time.sleep(random.randint(5, 8))

      titulo = driver.find_element(By.CSS_SELECTOR, '[data-testid="complaint-title"]').text
      lugar = driver.find_element(By.CSS_SELECTOR, '[data-testid="complaint-location"]').text
      data = driver.find_element(By.CSS_SELECTOR, '[data-testid="complaint-creation-date"]').text
      reclamacao_id = driver.find_element(By.CSS_SELECTOR, '[data-testid="complaint-id"]').text
      
      reclamacao_id = (reclamacao_id.split('ID: ')[1])

      parsed_data = time.strptime(data, '%d/%m/%Y às %H:%M')

      reclamacao_base = driver.find_element(By.CSS_SELECTOR, '.sc-lzlu7c-17.fRVYjv').text
      interacoes = driver.find_elements(By.CSS_SELECTOR, '[data-testid="complaint-interaction"]')

      interacoes_text = [i.text for i in interacoes]

      again = driver.find_element(By.CSS_SELECTOR, '.sc-uh4o7z-0.bprQlw:nth-child(2) > div').text
      rating = driver.find_element(By.CSS_SELECTOR, '.sc-uh4o7z-0.bprQlw:nth-child(3) > div').text

      rating = int(rating.split()[0])
      again = again == 'Sim'

      reclamacao = {
         'id': reclamacao_id,
         'titulo': titulo,
         'lugar': lugar,
         'data': parsed_data,
         'reclamacao_base': reclamacao_base,
         'interacoes': interacoes_text,
         'rating': rating,
         'again': again
      }

      reclamacoes.append(reclamacao)
      

   except:
      
      continue

print(reclamacoes)

   
# driver.quit()
   

